In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB

In [2]:
data = pd.read_csv("/Users/chenxu/Desktop/Crimes_-_2010.csv")
data.head()

,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,Ward,FBI Code,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Location
0,JC510649,01/01/2010 07:00:00 PM,033XX W WARNER AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,APARTMENT,False,False,1724,33.0,02,NaN,NaN,2010,NaN,NaN,NaN
1,JC510203,04/15/2010 08:00:00 AM,025XX W AUGUSTA BLVD,1120,DECEPTIVE PRACTICE,FORGERY,RESIDENCE,False,False,1211,26.0,10,NaN,NaN,2010,NaN,NaN,NaN
2,JC507834,06/01/2010 02:50:00 PM,102XX S AVENUE N,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,432,10.0,11,NaN,NaN,2010,NaN,NaN,NaN
3,JC502258,11/07/2010 04:15:00 PM,041XX W SCHOOL ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,1731,30.0,11,NaN,NaN,2010,NaN,NaN,NaN
4,JC490052,01/01/2010 12:00:00 AM,017XX W 48TH ST,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,APARTMENT,False,False,931,15.0,17,NaN,NaN,2010,NaN,NaN,NaN


### 查看数据概况

In [3]:
data.describe()

,Beat,Ward,X Coordinate,Y Coordinate,Year,Latitude,Longitude
count,370359.000000,370341.000000,3.699970e+05,3.699970e+05,370359.0,369997.000000,369997.000000
mean,1212.411744,22.746193,1.164287e+06,1.885372e+06,2010.0,41.841059,-87.672647
std,707.208183,13.697842,1.831233e+04,3.444941e+04,0.0,0.094852,0.065887
min,111.000000,1.000000,0.000000e+00,0.000000e+00,2010.0,36.619446,-91.686566
25%,624.000000,10.000000,1.152766e+06,1.858791e+06,2010.0,41.767903,-87.714441
50%,1112.000000,22.000000,1.165621e+06,1.890151e+06,2010.0,41.854298,-87.667468
75%,1733.000000,34.000000,1.176178e+06,1.909691e+06,2010.0,41.908123,-87.629210
max,2535.000000,50.000000,1.205119e+06,1.951499e+06,2010.0,42.022549,-87.524529


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370359 entries, 0 to 370358
Data columns (total 18 columns):
Case Number             370358 non-null object
Date                    370359 non-null object
Block                   370359 non-null object
IUCR                    370359 non-null object
Primary Type            370359 non-null object
Description             370359 non-null object
Location Description    370311 non-null object
Arrest                  370359 non-null bool
Domestic                370359 non-null bool
Beat                    370359 non-null int64
Ward                    370341 non-null float64
FBI Code                370359 non-null object
X Coordinate            369997 non-null float64
Y Coordinate            369997 non-null float64
Year                    370359 non-null int64
Latitude                369997 non-null float64
Longitude               369997 non-null float64
Location                369997 non-null object
dtypes: bool(2), float64(5), int64(2), objec

In [5]:
#查看每个字段不重复的变量有多少个，为挑选目标变量做准备
col_num = {}
for col in data.columns:
    col_num[col] = len(data[col].unique())
col_num

{'Case Number': 370331,
 'Date': 129442,
 'Block': 28736,
 'IUCR': 325,
 'Primary Type': 30,
 'Description': 304,
 'Location Description': 120,
 'Arrest': 2,
 'Domestic': 2,
 'Beat': 302,
 'Ward': 51,
 'FBI Code': 25,
 'X Coordinate': 56289,
 'Y Coordinate': 84050,
 'Year': 1,
 'Latitude': 175673,
 'Longitude': 175644,
 'Location': 175720}

### Arrest作为目标变量

In [6]:
data['Arrest'] = data['Arrest'].astype(int)
data.head()

,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,Ward,FBI Code,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Location
0,JC510649,01/01/2010 07:00:00 PM,033XX W WARNER AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,APARTMENT,0,False,1724,33.0,02,NaN,NaN,2010,NaN,NaN,NaN
1,JC510203,04/15/2010 08:00:00 AM,025XX W AUGUSTA BLVD,1120,DECEPTIVE PRACTICE,FORGERY,RESIDENCE,0,False,1211,26.0,10,NaN,NaN,2010,NaN,NaN,NaN
2,JC507834,06/01/2010 02:50:00 PM,102XX S AVENUE N,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,0,False,432,10.0,11,NaN,NaN,2010,NaN,NaN,NaN
3,JC502258,11/07/2010 04:15:00 PM,041XX W SCHOOL ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,0,False,1731,30.0,11,NaN,NaN,2010,NaN,NaN,NaN
4,JC490052,01/01/2010 12:00:00 AM,017XX W 48TH ST,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,APARTMENT,0,False,931,15.0,17,NaN,NaN,2010,NaN,NaN,NaN


### 查看Arrest与Primary Type的关系（可视化）

In [7]:
fbi_arrest_count = data.groupby(['Primary Type','Arrest']).agg({'Arrest':'count'}).unstack()
fbi_arrest_count.columns = [0,1]
fbi_arrest_count

,0,1
Primary Type,,
ARSON,456.0,66.0
ASSAULT,16485.0,5051.0
BATTERY,51125.0,14278.0
BURGLARY,25039.0,1383.0
CRIM SEXUAL ASSAULT,1124.0,245.0
CRIMINAL DAMAGE,38012.0,2641.0
CRIMINAL TRESPASS,2899.0,6502.0
DECEPTIVE PRACTICE,10445.0,2080.0
GAMBLING,NaN,927.0


In [8]:
from pyecharts.charts import Bar
from pyecharts import options as opts
import random
%matplotlib inline
attr = fbi_arrest_count.index.tolist()
v1 = fbi_arrest_count[0].tolist()
v2 = fbi_arrest_count[1].tolist()
#bar = Bar()
#bar.add("Arrest 0", attr, v1, is_label_show = True, xaxis_label_textsize = 6, xaxis_rotate = 90, is_stack=True)
#bar.add("Arrest 1", attr, v2, is_label_show = True, xaxis_label_textsize = 6,xaxis_rotate = 90,is_stack=True)
#bar.render("bar.html")
bar = (
        Bar()
        .add_xaxis(attr)
        .add_yaxis("Not Arrest", v1, stack="stack1",color="black")
        .add_yaxis("Arrest", v2,stack="stack1",color="red")
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="Primary Type Count By FBI Code"),
                         xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=90,font_size=10))
                         
        )
    )
bar.render("bar.html")

'/Users/chenxu/Desktop/bar.html'

### 数据处理、缺失值填充

In [9]:
from sklearn.preprocessing import LabelEncoder
feat_col = [col for col in data.columns if col not in ['Case Number','Arrest']]
obj_col = data[feat_col].dtypes[data[feat_col].dtypes == 'object'].index.tolist()
for col in obj_col:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    
    
for col in feat_col:
    data[col] = data[col].fillna(data[col].mode()[0])
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370359 entries, 0 to 370358
Data columns (total 18 columns):
Case Number             370358 non-null object
Date                    370359 non-null int64
Block                   370359 non-null int64
IUCR                    370359 non-null int64
Primary Type            370359 non-null int64
Description             370359 non-null int64
Location Description    370359 non-null int64
Arrest                  370359 non-null int64
Domestic                370359 non-null bool
Beat                    370359 non-null int64
Ward                    370359 non-null float64
FBI Code                370359 non-null int64
X Coordinate            370359 non-null float64
Y Coordinate            370359 non-null float64
Year                    370359 non-null int64
Latitude                370359 non-null float64
Longitude               370359 non-null float64
Location                370359 non-null int64
dtypes: bool(1), float64(5), int64(11), object(1)
m

In [10]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
train_x,val_x,train_y,val_y = train_test_split(data[feat_col],data['Arrest'],test_size=0.8,random_state=2019)
'done'

'done'

In [11]:
clf = GaussianNB()
clf.fit(train_x,train_y)
pre_val = clf.predict_proba(val_x)[::,1]
print('roc is',roc_auc_score(val_y,pre_val))

roc is 0.7514373067936143
